In [ ]:
# Install PyLLSM5DTools in your env
# You can skip this step if you've already installed it in the env you are using for this notebook
%pip install --no-binary :all: --no-cache-dir PyLLSM5DTools

In [ ]:
# You can set the folder where our demo dataset will be downloaded to
# By default it will be the Downloads folder in your home directory
# I.E. Setting llsm5dtools_demo_dataset_install_dir = "~/Downloads/" will install
# the dataset to ~/Downloads/LLSM5DTools_demo_cell_image_dataset/
import os
import subprocess
import urllib.request

# Change this variable to your desired download location
llsm5dtools_demo_dataset_install_dir = "~/Downloads/"

llsm5dtools_demo_dataset_install_dir = os.path.expanduser(llsm5dtools_demo_dataset_install_dir)
llsm5dtools_demo_dataset_install_dir = os.path.join(llsm5dtools_demo_dataset_install_dir,'')
llsm5dtools_demo_dataset_url = "https://www.dropbox.com/scl/fi/dszpkbrp44c0qpagr1wxl/LLSM5DTools_demo_cell_image_dataset.tar?rlkey=p4pll0l42xt6fekm47m8jzni7&dl=1"
llsm5dtools_demo_dataset_dir_loc = os.path.join(llsm5dtools_demo_dataset_install_dir, "LLSM5DTools_demo_cell_image_dataset/")
llsm5dtools_demo_dataset_tar_loc = os.path.join(llsm5dtools_demo_dataset_install_dir, "LLSM5DTools_demo_cell_image_dataset.tar")
os.makedirs(os.path.dirname(llsm5dtools_demo_dataset_install_dir), exist_ok=True)
if not os.path.exists(llsm5dtools_demo_dataset_dir_loc):
    print("Downloading the demo dataset")
    urllib.request.urlretrieve(llsm5dtools_demo_dataset_url, llsm5dtools_demo_dataset_tar_loc)
    print("Demo dataset downloaded successfully")
    print("Untarring the demo dataset")
    process = subprocess.Popen(
    f"tar -xf \"{llsm5dtools_demo_dataset_tar_loc}\" -C \"{llsm5dtools_demo_dataset_install_dir}\"", shell=True)
    process.wait()
    print("Demo dataset untarred successfully")
    print("Removing the tar file")
    os.remove(llsm5dtools_demo_dataset_tar_loc)
    print("Finished!")
    print(f"The demo dataset is located here: {llsm5dtools_demo_dataset_dir_loc}")
else:
    print(f"The demo dataset is already located here: {llsm5dtools_demo_dataset_dir_loc}")


In [ ]:
# 

In [ ]:
# Example for running a DSR on our demo dataset
from PyLLSM5DTools import XR_deskew_rotate_data_wrapper

# Set your data path to our demo dataset
dataPath = '~/Downloads/LLSM5DTools_demo_cell_image_dataset/'
dataPath_exps = [dataPath]

# run deskew if true
Deskew = True
# run rotation if true
Rotate = True
# use combined processing if true. Note: if you only need deskew without rotation, set both Rotate and DSRCombined
# as false.
DSRCombined = True
# xy pixel size
xyPixelSize = 0.108
# z scan step size
dz = 0.3
# Skew angle
SkewAngle = 32.45
# if true, objective scan; otherwise, sample scan
ObjectiveScan = False
# scan direction
Reverse = True
# channel patterns to map the files for processing
ChannelPatterns = ['CamA', 'CamB']

# flat field related parameters
# disable flat field for this demo
LLFFCorrection = False
# lower bound cap for flat field
LowerLimit = 0.4
# constant offset after flat field correction
constOffset = 1
# flat field image paths
LSImagePaths = [
    dataPath + 'FF/averaged/ff_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0096911533msecAbs_000x_000y_000z_0017t.tif',
    dataPath + 'FF/averaged/ff_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0096911533msecAbs_000x_000y_000z_0017t.tif']

# background image paths
BackgroundPaths = [dataPath + 'FF/KorraFusions/AVG_DF400_CamA_10ms.tif',
                   dataPath + 'FF/KorraFusions/AVG_DF400_CamB_10ms.tif']

# if true, use large scale processing pipeline (split, process, and then merge)
largeFile = False
# true if input is in zarr format
zarrFile = False
# save output as zarr if true
saveZarr = False
# block size to save the result
blockSize = [256, 256, 256]
# save output as uint16 if true
Save16bit = True

# use slurm cluster if true, otherwise use the local machine (master job)
parseCluster = False
# use master job for task computing or not.
masterCompute = True
# configuration file for job submission
configFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

XR_deskew_rotate_data_wrapper(dataPath_exps, Deskew=Deskew, Rotate=Rotate,
                              DSRCombined=DSRCombined, xyPixelSize=xyPixelSize, dz=dz, SkewAngle=SkewAngle,
                              ObjectiveScan=ObjectiveScan, Reverse=Reverse, ChannelPatterns=ChannelPatterns,
                              LLFFCorrection=LLFFCorrection, LowerLimit=LowerLimit, constOffset=constOffset,
                              LSImagePaths=LSImagePaths, BackgroundPaths=BackgroundPaths, largeFile=largeFile,
                              zarrFile=zarrFile, saveZarr=saveZarr, blockSize=blockSize, Save16bit=Save16bit,
                              parseCluster=parseCluster, masterCompute=masterCompute, ConfigFile=configFile,
                              mccMode=mccMode)


In [ ]:
# Example for running a deconvolution on our demo dataset
from PyLLSM5DTools import XR_decon_data_wrapper

# Set your data path to our demo dataset
dataPath = '~/Downloads/LLSM5DTools_demo_cell_image_dataset/'
dataPaths = [dataPath]
rt = dataPath
# xy pixel size in um
xyPixelSize = 0.108
# z step size
dz = 0.3
# scan direction
Reverse = True
# psf z step size (we assume xyPixelSize also apply to psf)
dzPSF = 0.3

# if true, check whether image is flipped in z using the setting files
parseSettingFile = False

# channel patterns for the channels, the channel patterns should map the
# order of PSF filenames.
ChannelPatterns = {'CamB_ch0'}

# psf path
psf_rt = rt
PSFFullpaths = [psf_rt + 'PSF/488_2_c.tif']

# RL method
RLmethod = 'omw'
# wiener filter parameter
# alpha parameter should be adjusted based on SNR and data quality.
# typically 0.002 - 0.01 for SNR ~20; 0.02 - 0.1 or higher for SNR ~7
wienerAlpha = 0.005
# OTF thresholding parameter
OTFCumThresh = 0.9
# true if the PSF is in skew space
skewed = True
# deconvolution result path string (within dataPath)
deconPathstr = 'matlab_decon_omw'

# background to subtract
Background = 100
# number of iterations
DeconIter = 2
# decon to 80 iterations (not use the criteria for early stop)
fixIter = True
# erode the edge after decon for number of pixels.
EdgeErosion = 8
# save as 16bit, if false, save to single
Save16bit = True
# use zarr file as input, if false, use tiff as input
zarrFile = False
# number of cpu cores
cpusPerTask = 24
# use cluster computing for different images
parseCluster = False
# set it to true for large files that cannot be fitted to RAM/GPU, it will
# split the data to chunks for deconvolution
largeFile = False
# use GPU for deconvolution
GPUJob = False
# if true, save intermediate results every 5 iterations.
debug = False
# config file for the master jobs that runs on CPU node
ConfigFile = ''
# config file for the GPU job scheduling on GPU node
GPUConfigFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

# Step 3.2: run the deconvolution with given parameters.
# the results will be saved in matlab_decon under the dataPaths.
# the next step is deskew/rotate (if in skewed space for x-stage scan) or
# rotate (if objective scan) or other processings.

# result folder:
# {destPath}/LLSM5DTools_demo_cell_image_dataset/matlab_decon_omw/

XR_decon_data_wrapper(dataPaths, deconPathstr=deconPathstr, xyPixelSize=xyPixelSize,
                      dz=dz, Reverse=Reverse, ChannelPatterns=ChannelPatterns, psfFullpaths=PSFFullpaths,
                      dzPSF=dzPSF, parseSettingFile=parseSettingFile, RLMethod=RLmethod,
                      wienerAlpha=wienerAlpha, OTFCumThresh=OTFCumThresh, skewed=skewed,
                      Background=Background, CPPdecon=False, CudaDecon=False, DeconIter=DeconIter,
                      fixIter=fixIter, EdgeErosion=EdgeErosion, Save16bit=Save16bit,
                      zarrFile=zarrFile, parseCluster=parseCluster, largeFile=largeFile,
                      GPUJob=GPUJob, debug=debug, cpusPerTask=cpusPerTask, ConfigFile=ConfigFile,
                      GPUConfigFile=GPUConfigFile, mccMode=mccMode)

In [ ]:
# Example for running a skewed space stitch on our demo dataset
from PyLLSM5DTools import XR_matlab_stitching_wrapper

# Set your data path to our demo dataset
dataPath = "~/Downloads/LLSM5DTools_demo_cell_image_dataset/"
dataPaths = [dataPath]

# image list path: csv file
# if not available, run stitch_generate_imagelist_from_encoder(dataPath, dz)
ImageListFullpath = [dataPath+"ImageList_from_encoder.csv"]

# stitch in DS space
DS = False
# stitch in Deskew/rotated space, if both DS and DSR false, stitch in skewed space
DSR = False

# skew angle
SkewAngle = 32.45

# scan direction. 
Reverse = True

# resolution [xyPixelsize, dz]
Resolution = [0.108, 0.30]

# check the setting files to see if a tile is flipped (bidirectional scan)
parseSettingFile = False

# axis order
axisOrder = '-x,y,z'

# stitch blending method, 'none': no blending, 'feather': feather blending
BlendMethod = 'feather'

# stitch dir string, inside dataPath
stitchResultDir = 'matlab_stitch'

# cross correlation registration, if false, directly stitch by the coordinates.
xcorrShift = True

# if true, only stitch first time point
onlyFirstTP = False

# stitch pipeline, no need to change, zarr pipeline is the mostly used one.
stitchPipeline = 'zarr'

# xcorr registration on the primary channel, and other channels uses the
# registration informaion from the primary channel
PrimaryCh = 'CamB_ch0'

# channels to stitch
ChannelPatterns = ['CamA_ch0', 'CamB_ch0']

# if true, save result as 16bit, if false, save as single
Save16bit = True

#########################################################################
# parameters below are used in special cases or for advanced fine-tuning

# resample data, if empty, stitch in original resolution. If a 1X3 array,
# resample data and then stitch, mostly used for initial inspection
resample = []

# resample type: isotropic, xy_isotropic; effective when 'resample' is
# not set. Keep it as 'isotropic'. 
resampleType = 'isotropic'

# max allowed shift (in pixel) in xy axes between neighboring tiles in xcorr registration. 
xyMaxOffset = 150

# max allowed shift in z (in pixel) axis between neighboring tiles in xcorr registration. 
zMaxOffset = 150

# downsampling factors for overlap regions to calculate xcorr in xcorr registration. 
# with larger downsamplinf factors, the faster the xcorr computing, but
# the lower accuracy will be. 
xcorrDownsample = [2, 2, 2]

# crop input tile before any processing, empty (no crop) or a 1X6 array [ymin, xmin, zmin, ymax, xmax, zmax]
InputBbox = []

# crop input tile after processing, empty (no crop) or a 1X6 array [ymin, xmin, zmin, ymax, xmax, zmax]
tileOutBbox = []

# chunk size in zarr
blockSize = [256, 256, 256]

# Currently, flat field correction is disabled in the demo
processFunPath = ''

# tile offset: counts add to the image, used when the image background is
# 0, to differentiate between image background and empty space. 
TileOffset = 0

# use slurm cluster if true, otherwise use the local machine (master job)
parseCluster = False
# use master job for task computing or not. 
masterCompute = True
# configuration file for job submission
configFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = False


# Step 2: run the stitching with given parameters. 
# the stitched results will be the zarr files in 'matlab_stitch_xcorr_feather_skewed_zarr'
# They are in skewed space, the next step is deconvolution/deskew rotation,
# or just deskew rotation. 


XR_matlab_stitching_wrapper(dataPaths, ImageListFullpath,
    DS=DS, DSR=DSR, SkewAngle=SkewAngle, Reverse=Reverse, axisOrder=axisOrder,
    xcorrShift=xcorrShift, ChannelPatterns=ChannelPatterns, PrimaryCh=PrimaryCh,
    blockSize=blockSize, TileOffset=TileOffset, resampleType=resampleType,
    resample=resample, Resolution=Resolution, BlendMethod=BlendMethod,
    resultDir=stitchResultDir, onlyFirstTP=onlyFirstTP, xyMaxOffset=xyMaxOffset,
    zMaxOffset=zMaxOffset, xcorrDownsample=xcorrDownsample, InputBbox=InputBbox,
    tileOutBbox=tileOutBbox, pipeline=stitchPipeline, parseSettingFile=parseSettingFile,
    processFunPath=processFunPath, Save16bit=Save16bit, parseCluster=parseCluster,
    masterCompute=masterCompute, configFile=configFile, mccMode=mccMode)

In [ ]:
# Example for running a crop on our demo dataset
from PyLLSM5DTools import XR_crop_dataset

# Set your data path to our demo dataset
dataPath = '~/Downloads/LLSM5DTools_demo_cell_image_dataset/'
XR_crop_dataset([dataPath], [f"{dataPath}/crop"], [1, 1, 1, 100, 100, 100])